In [152]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
from IPython.display import display, Markdown
import tiktoken
import os
import json
import random
import subprocess

load_dotenv(find_dotenv())

True

In [153]:
import os
from anthropic import Anthropic

client = Anthropic(
    # This is the default and can be omitted
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)


In [154]:
OpenAI_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
Anthropic_client = Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

In [155]:
def get_n_tokens(text):
    enc = tiktoken.encoding_for_model("gpt-4o")
    return len(enc.encode(text))

def get_openai_answer(instructions, conversation_history, new_prompt, json_output=False):
    if len(conversation_history)==0:
        conversation_history.append({"role": "system", "content": instructions})
    conversation_history.append({"role": "user", "content": new_prompt})

    if json_output:
        completion = OpenAI_client.chat.completions.create(
          model="gpt-4o",
          messages=conversation_history,
          response_format={"type": "json_object"}
        )
    else:
        completion = OpenAI_client.chat.completions.create(
          model="gpt-4o",
          messages=conversation_history
        )

    conversation_history.append({"role": "assistant", "content": completion.choices[0].message.content})
    
    return completion.choices[0].message.content, conversation_history


def get_claude_answer(instructions, conversation_history, new_prompt):
    conversation_history.append({"role": "user", "content": new_prompt})
        
    # Call the Claude API with the updated conversation history
    message = client.messages.create(
        max_tokens=2000,
        system=instructions,
        messages=conversation_history,
        model="claude-3-5-sonnet-20240620",
    )
    
    # Append the Claude's response to the conversation history
    conversation_history.append({"role": "assistant", "content": message.content[0].text})
    
    return message.content[0].text, conversation_history


def read_file(file_path):
    try:
        with open(file_path, 'r') as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return "File not found."

def read_all_files_in_folder(folder_path):
    contents_dict = {}
    for filename in os.listdir(folder_path):
        if filename[-4:] == ".sol":
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path):
                with open(file_path, 'r') as file:
                    contents = file.read()
                    contents_dict[filename] = contents
    return contents_dict

def get_cost_dollars(instructions, answer):
    input_1k_tokens = 0.005
    output_1k_tokens = 0.015
    return input_1k_tokens * get_n_tokens(instructions)/1000 + output_1k_tokens * get_n_tokens(answer)/1000

def render_markdown(text):
    display(Markdown(text))

def sample_dict(dictionary, sample_size):
    if sample_size > len(dictionary):
        raise ValueError("Sample size cannot be larger than the dictionary size")
    
    sampled_keys = random.sample(list(dictionary.keys()), sample_size)
    sampled_dict = {key: dictionary[key] for key in sampled_keys}
    
    return sampled_dict

def save_to_sol(content, folder_path, file_name):
    # Ensure the file has the .sol extension
    if not file_name.endswith('.sol'):
        file_name += '.sol'

    # Combine the folder path and file name to get the full file path
    file_path = os.path.join(folder_path, file_name)
    
    # Open the file in write mode and write the content
    with open(file_path, 'w') as file:
        file.write(content)

    print(f"File {file_path} has been created and saved successfully.")


def compile_contract(file_to_build):

    command = f"forge build src/generated/{file_to_build}"
    working_directory = "/Users/miquel/Desktop/git/miqlar/cook-some-hooks/foundry_hook_playground"

    # Run the command in the specified directory
    result = subprocess.run(command, shell=True, capture_output=True, text=True, cwd=working_directory)

    # Return stdout, stderr, and returncode
    return result.stdout, result.stderr, result.returncode


def markdown_to_text(text):
    spdx_str = "// SPDX-License-Identifier"
    index = text.find(spdx_str)
    if index != -1:
        # Strip leading and trailing whitespace, then remove trailing backticks
        return text[index:].strip().rstrip('`')
    else:
        # If the SPDX string is not found, strip whitespace and remove trailing backticks
        return text.strip().rstrip('`')
        


In [160]:
file_path = 'instructions.txt'
instructions = read_file(file_path)

folder_path = '../foundry_hook_playground/src/examples'
file_to_code = read_all_files_in_folder(folder_path)

with open("hook_examples.json", 'r') as file:
    hook_examples_json = json.load(file)

In [162]:
len(file_to_code.keys())

14

In [163]:
rag_instructions = f"""
You have a JSON object mapping file names to summaries:

{hook_examples_json}

You will receive a user prompt and your task is to determine which Solidity files are most relevant to the prompt.

### Instructions:

1. **Analyze the user prompt**: Understand the key concepts and requirements from the user's prompt.
2. **Match with summaries**: Compare these key concepts with the summaries provided in the JSON object.
3. **Evaluate relevance**: Assign a confidence level to each file based on how well it matches the user's prompt. Use the following confidence levels:
   - High: Very relevant to the prompt
   - Medium: Somewhat relevant to the prompt
   - Low: Slightly relevant to the prompt

### Output Format:

Return a ONLY a JSON object with the top 5 most relevant files based on your evaluation. Each entry should have the following format:
file_name: high/medium/low

Return ONLY a JSON with 5 entries
"""

In [164]:
prompt = "I want a hook that does kyc so that if you dont have a certain nft you cant trade, and also it only allows to trade when the hour is pair"

rag_answer, conversation_history = get_openai_answer(rag_instructions, [], prompt, json_output=True)

In [166]:
eval(rag_answer)

{'KYCHook.sol': 'high',
 'ERC721OwnershipHook.sol': 'high',
 'TradingHours.sol': 'high',
 'WhiteListHook.sol': 'medium',
 'MultiSigSwapHook.sol': 'low'}

In [167]:
final_instructions = instructions
counter = 1

for file in eval(rag_answer).keys():
    if file not in list(hook_examples_json.keys()):
        print(f"File {file} does not exist!")
    else:
        final_instructions += f"""----------\nHOOK EXAMPLE {counter}:\n\n
        SUMMARY: {hook_examples_json[file]}\n
        CODE:\n {file_to_code[file]}\n\n\n------------------\n\n\n"""
        counter+=1

final_instructions+="OUTPUT: ONLY Solidity code, nothing else - no explanations, summaries or descriptions. ONLY working Solidity code, WITH comments."

In [168]:
get_n_tokens(final_instructions)

6476

In [169]:
attempt_counter = 0
conversation_history = []
returncode =- 1

while (attempt_counter<5) and (returncode!=0):
    file_name = f"test_{attempt_counter}.sol"
    #answer, conversation_history = get_claude_answer(final_instructions, conversation_history, prompt)
    answer, conversation_history = get_openai_answer(final_instructions, conversation_history, prompt)
    answer = markdown_to_text(answer)
    save_to_sol(answer, "../foundry_hook_playground/src/generated/", file_name)
    stdout, stderr, returncode = compile_contract(file_name)
    prompt = "I get this error when compiling the contract: \n\n"+stderr+"\n\nOUTPUT: Only the fixed solidity code"
    if (returncode)!=0:
        print(stderr)
        print("-----------------------")
    else:
        print("\U0001F389 \U0001F389 Contract compiled! \U0001F389 \U0001F389")
    attempt_counter+=1

File ../foundry_hook_playground/src/generated/test_0.sol has been created and saved successfully.
🎉 🎉 Contract compiled! 🎉 🎉
